## Database Analysis
This notebook begins by outlining a scenario that anyone who works for an entity that collects customer/client data will encounter. This use-case is the possibility of a new batch of incoming electronic data containing errors. 


There are some steps and computations I'll be performing on this:
1. Identify possible problems with the data
2. Attempt to mitigate the issues as much as possible
3. Report all discrepancies with detailed logs of why a row is labeled as erroneous. 

The Python script will allow the user to set parameters or arguments easily (rather than going into this notebook and manually changing everything). 

### Configuration

In [17]:
import pandas as pd

In [18]:
database_df = pd.read_csv('data/database.csv') # replace this file with whatever the database file is
print(database_df.head())

                                   guid                                 altid  \
0  986FC58A-3B13-7DEE-ECDA-95E2388AE5FD  1D265437-1314-4EA7-2B32-26D30A495809   
1  082789E2-9D3A-DA84-2086-DEDEC83F46C2  2167E809-4F99-EA36-A306-177CA6B8CBD5   
2  75261691-7A59-E6E3-D45F-D71DCDE9B64E  CEE49EDD-5337-EA8D-E5D9-695EA794675E   
3  6F2AE798-B7B5-43C2-43D3-592B1109FA23  2BA41BA5-BD91-2C6E-BDBA-7AEBEBCA4A8B   
4  C1DFC1C5-E1AE-6C3C-B896-6AC906D78B75  B88D594D-D82B-FF96-6A13-BE3F5EA8C09E   

  first_name   last_name                        address     zip  \
0      Daryl  Valenzuela         650-7555 Pharetra. Ave  665818   
1   Chiquita        Cook  P.O. Box 445, 1985 Tellus Av.   61683   
2     Hayden     Knowles    Ap #209-2681 Ultrices. Road  278988   
3      Karen  Fitzgerald             239-4983 Metus St.  661343   
4   Harrison    Bradshaw  P.O. Box 672, 3567 Lorem, St.   13301   

                           region        lat         lon  
0  Brussels Hoofdstedelijk Gewest  41.573768 -169.5

In [19]:
target_df = pd.read_csv('data/target.csv') # replace this file with whatever the database file is
print(target_df.head())

                                   guid                                 altid  \
0  986FC58A-3B13-7DEE-ECDA-95E2388AE5FD  1D265437-1314-4EA7-2B32-26D30A495809   
1  082789E2-9D3A-DA84-2086-DEDEC83F46C2  2167E809-4F99-EA36-A306-177CA6B8CBD5   
2  75261691-7A59-E6E3-D45F-D71DCDE9B64E  CEE49EDD-5337-EA8D-E5D9-695EA794675E   
3  6F2AE798-B7B5-43C2-43D3-592B1109FA23  2BA41BA5-BD91-2C6E-BDBA-7AEBEBCA4A8B   
4  C1DFC1C5-E1AE-6C3C-B896-6AC906D78B75  B88D594D-D82B-FF96-6A13-BE3F5EA8C09E   

  first_name   last_name                        address     zip  \
0      Daryl  Valenzuela         650-7555 Pharetra. Ave  665818   
1      Daryl  Valenzuela  P.O. Box 445, 1985 Tellus Av.   61683   
2      Daryl  Valenzuela    Ap #209-2681 Ultrices. Road  278988   
3      Karen  Fitzgerald             239-4983 Metus St.  661343   
4   Harrison    Bradshaw  P.O. Box 672, 3567 Lorem, St.   13301   

                           region        lat         lon  
0  Brussels Hoofdstedelijk Gewest  41.573768 -169.5

In [21]:
print(target_df.isin(database_df))

      guid  altid  first_name  last_name  address    zip  region    lat    lon
0     True   True        True       True     True   True    True   True   True
1     True   True       False      False     True   True    True   True   True
2     True   True       False      False     True   True    True   True   True
3     True   True        True       True     True   True    True   True   True
4     True   True        True       True     True   True    True   True   True
..     ...    ...         ...        ...      ...    ...     ...    ...    ...
495  False  False       False      False    False  False   False  False  False
496  False  False       False      False    False  False   False  False  False
497  False  False       False      False    False  False   False  False  False
498  False  False       False      False    False  False   False  False  False
499  False  False       False      False    False  False   False  False  False

[500 rows x 9 columns]


### __Data Validation__

#### Missing Data
Missing in this case means rows containing elements that are not present in the database using an __outer join__. For more information on this SQL style command: https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas.merge

In [22]:
missing_df = pd.merge(database_df, target_df, how='outer', indicator=True)
missing_df = missing_df[missing_df._merge != 'both']
missing_df

,guid,altid,first_name,last_name,address,zip,region,lat,lon,_merge
1,082789E2-9D3A-DA84-2086-DEDEC83F46C2,2167E809-4F99-EA36-A306-177CA6B8CBD5,Chiquita,Cook,"P.O. Box 445, 1985 Tellus Av.",61683,Phú Thọ,-35.525666,44.406580,left_only
2,75261691-7A59-E6E3-D45F-D71DCDE9B64E,CEE49EDD-5337-EA8D-E5D9-695EA794675E,Hayden,Knowles,Ap #209-2681 Ultrices. Road,278988,Xīnán,-68.865114,62.478500,left_only
448,082789E2-9D3A-DA84-2086-DEDEC83F46C2,2167E809-4F99-EA36-A306-177CA6B8CBD5,Daryl,Valenzuela,"P.O. Box 445, 1985 Tellus Av.",61683,Phú Thọ,-35.525666,44.406580,right_only
449,75261691-7A59-E6E3-D45F-D71DCDE9B64E,CEE49EDD-5337-EA8D-E5D9-695EA794675E,Daryl,Valenzuela,Ap #209-2681 Ultrices. Road,278988,Xīnán,-68.865114,62.478500,right_only
450,BBB8F595-DBCB-D73C-1B5D-A3550E9DFD25,39ECC637-112E-CC3C-849A-A5B92F069573,Kelly,Mayo,Ap #994-3283 Vitae Ave,54851,Biobío,36.922785,-67.639421,right_only
451,6641EA45-3AB1-1159-659C-E24B3525AD71,AEBB7B86-4C1A-C321-D8A5-746657A1B434,Buffy,Hanson,"P.O. Box 777, 7780 Vitae St.",392850,North-East Region,-67.966309,21.075739,right_only
452,3B067CCE-DADD-CB9E-5CEB-934D17E37A82,2CBC31B1-DD9F-48F3-6788-461205C94B1E,Dorian,Daniels,410-6467 Praesent Street,87318,Veracruz,68.826459,-86.346122,right_only
453,4A5D1DE0-6462-3385-C266-519ED8C7E2C2,77C964AC-5B12-0B92-A36B-58887DC037B9,Dawn,Parker,832 Nunc Rd.,5511,Utah,70.454556,-122.581567,right_only
454,770948EF-8EE9-0461-C695-0A63C9A70793,5E996CE8-BD89-874D-B3C5-537C73FE84A0,Maya,West,Ap #292-8014 Risus. Ave,56791,Queensland,-84.188598,16.038648,right_only
455,C6928884-037E-01FC-194D-742A1D75BAF3,2796D85F-A08D-3D59-BBA3-7389E546CB68,Raphael,Lowery,"7940 Pede, Av.",16-262,Basilicata,37.548250,-95.640124,right_only


Before we add these to our database, we should make sure they aren't duplicates. In this case, we can validate their identities using some form of identification (like this datasets GUID). Depending on different scenarios, a business could require every client to have a different ```guid```, or in other cases they might have overlapping ID's. For our use case we will assume that ID's should __not__ be duplicated.

Luckily ```pandas``` supports an easy way for us to check if values in one column are present in another dataframe using ```isin```.

In [23]:
missing_df.guid.isin(database_df.guid).astype(int)

1      1
2      1
448    1
449    1
450    0
451    0
452    0
453    0
454    0
455    0
456    0
457    0
458    0
459    0
460    0
461    0
462    0
463    0
464    0
465    0
466    0
467    0
468    0
469    0
470    0
471    0
472    0
473    0
474    0
475    0
476    0
477    0
478    0
479    0
480    0
481    0
482    0
483    0
484    0
485    0
486    0
487    0
488    0
489    0
490    0
491    0
492    0
493    0
494    0
495    0
496    0
497    0
498    0
499    0
500    0
501    0
Name: guid, dtype: int32

We now know that some of these ```guid``` are duplicates. Let's see if these are the same individuals or not. There are many use-cases in business where a client could have changed their address or name, resulting in the database having conflicting data.